In [2]:
%pip install osmnx geopandas shapely rasterio rasterstats numpy


Note: you may need to restart the kernel to use updated packages.


In [3]:
import osmnx as ox
import geopandas as gpd
from rasterstats import zonal_stats
import rasterio
import json
import numpy as np
from shapely.geometry import Point


In [4]:
import os
os.getcwd()  # confirm working directory
os.chdir("../")


In [5]:
graph_path = "data/processed/roads_base.graphml"
G = ox.load_graphml(graph_path)

edges = ox.graph_to_gdfs(G, nodes=False, edges=True)
edges.head()


osmid      highway lanes                  name  \
u        v          key                                                        
17327095 248007842  0      32261256  residential     2         2nd Main Road   
         305154531  0    1367650597    secondary     2        9th Cross Road   
         5354313543 0    1367650758  residential   NaN         2nd Main Road   
17327139 443224605  0     111814615      primary     2  Mahayogi Vemana Road   
         309593057  0      28186701  residential   NaN        7th Cross Road   

                         oneway reversed      length  \
u        v          key                                
17327095 248007842  0      True    False  244.091315   
         305154531  0      True    False   29.723619   
         5354313543 0     False     True    5.961520   
17327139 443224605  0      True    False   44.630750   
         309593057  0     False    False   50.303992   

                                                                  geometry  \
u        v          key                                                      
17327095 248007842  0    LINESTRING (77.59872 12.91054, 77.59848 12.91272)   
         305154531  0    LINESTRING (77.59872 12.91054, 77.59899 12.91056)   
         5354313543 0    LINESTRING (77.59872 12.91054, 77.59873 12.91049)   
17327139 443224605  0    LINESTRING (77.62407 12.93497, 77.62437 12.93524)   
         309593057  0    LINESTRING (77.62407 12.93497, 77.62402 12.935...   

                        maxspeed  ref bridge width access junction tunnel  \
u        v          key                                                     
17327095 248007842  0        NaN  NaN    NaN   NaN    NaN      NaN    NaN   
         305154531  0        NaN  NaN    NaN   NaN    NaN      NaN    NaN   
         5354313543 0        NaN  NaN    NaN   NaN    NaN      NaN    NaN   
17327139 443224605  0        NaN  NaN    NaN   NaN    NaN      NaN    NaN   
         309593057  0        NaN  NaN    NaN   NaN    NaN      NaN    NaN   

                        est_width area service  
u        v          key                         
17327095 248007842  0         NaN  NaN     NaN  
         305154531  0         NaN  NaN     NaN  
         5354313543 0         NaN  NaN     NaN  
17327139 443224605  0         NaN  NaN     NaN  
         309593057  0         NaN  NaN     NaN

In [6]:
os.getcwd()

'/home/kuldeep-chamoli/Desktop/GIS/GIS'

In [7]:
!find /home -type f -name "*ndvi*" 2>/dev/null


/home/kuldeep-chamoli/Desktop/GIS/GIS/src/data processing/ndvi_extractor.py
/home/kuldeep-chamoli/Desktop/GIS/GIS/data/raw/sentinel_ndvi/ndvi_bengaluru_2024.tif
/home/kuldeep-chamoli/Desktop/GIS/GIS/data/raw/sentinel_ndvi/ndvi_bengaluru_2024.tiff
/home/kuldeep-chamoli/Desktop/GIS/GIS/data/processed/edges_ndvi_aqi.geojson
/home/kuldeep-chamoli/Desktop/GIS/GIS/data/processed/edges_ndvi_aqi.csv
/home/kuldeep-chamoli/Desktop/GIS/GIS/notebooks/02_compute_ndvi_gee.ipynb
/home/kuldeep-chamoli/Desktop/GIS/GIS/notebooks/04_merge_ndvi_aqi_with_roads.ipynb


In [8]:
import os
os.getcwd()


'/home/kuldeep-chamoli/Desktop/GIS/GIS'

In [9]:
import rasterio
import numpy as np

ndvi_tif_path = "data/raw/sentinel_ndvi/ndvi_bengaluru_2024.tif"

print("Loading NDVI raster...")
with rasterio.open(ndvi_tif_path) as src:
    ndvi = src.read(1)
    transform = src.transform
    ndvi_nodata = src.nodata if src.nodata is not None else -9999

print("Computing centroids...")
edges["centroid"] = edges.geometry.centroid

def sample_ndvi(point):
    try:
        x, y = point.x, point.y
        col, row = ~transform * (x, y)
        row, col = int(row), int(col)

        if 0 <= row < ndvi.shape[0] and 0 <= col < ndvi.shape[1]:
            value = ndvi[row, col]
            if value == ndvi_nodata or np.isnan(value):
                return np.nan
            return float(value)
    except:
        return np.nan
    return np.nan

print("Sampling NDVI (fast centroid method)...")
edges["ndvi"] = edges["centroid"].apply(sample_ndvi)
edges["ndvi"] = edges["ndvi"].fillna(0)


Loading NDVI raster...
Computing centroids...


/tmp/ipykernel_6466/3886547529.py:13: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  edges["centroid"] = edges.geometry.centroid


Sampling NDVI (fast centroid method)...


In [13]:
with open("data/raw/aqi_meteo.json") as f:
    aqi_data = json.load(f)

df_hourly = aqi_data["hourly"]


In [14]:
import pandas as pd

df = pd.DataFrame(df_hourly)
df["pm2_5"] = pd.to_numeric(df["pm2_5"], errors="coerce")

avg_pm25 = df["pm2_5"].mean()

print("Average PM2.5 for Bengaluru:", avg_pm25)

# Assign same AQI to all edges (city-wide pollution)
edges["aqi"] = avg_pm25


Average PM2.5 for Bengaluru: 13.73394495412844


In [16]:
# Restore u, v, key from the MultiIndex
edges["u"] = edges.index.get_level_values(0)
edges["v"] = edges.index.get_level_values(1)
edges["key"] = edges.index.get_level_values(2)


In [17]:
edges = edges[["u", "v", "key", "geometry", "length", "ndvi", "aqi"]]
edges.head()


u           v  key  \
u        v          key                              
17327095 248007842  0    17327095   248007842    0   
         305154531  0    17327095   305154531    0   
         5354313543 0    17327095  5354313543    0   
17327139 443224605  0    17327139   443224605    0   
         309593057  0    17327139   309593057    0   

                                                                  geometry  \
u        v          key                                                      
17327095 248007842  0    LINESTRING (77.59872 12.91054, 77.59848 12.91272)   
         305154531  0    LINESTRING (77.59872 12.91054, 77.59899 12.91056)   
         5354313543 0    LINESTRING (77.59872 12.91054, 77.59873 12.91049)   
17327139 443224605  0    LINESTRING (77.62407 12.93497, 77.62437 12.93524)   
         309593057  0    LINESTRING (77.62407 12.93497, 77.62402 12.935...   

                             length      ndvi        aqi  
u        v          key                                   
17327095 248007842  0    244.091315  0.333333  13.733945  
         305154531  0     29.723619  0.401188  13.733945  
         5354313543 0      5.961520  0.490335  13.733945  
17327139 443224605  0     44.630750  0.369334  13.733945  
         309593057  0     50.303992  0.355148  13.733945

In [19]:
print("Preparing for GeoJSON export...")

# Drop centroid if present (GeoJSON supports only 1 geometry field)
edges = edges.drop(columns=["centroid"], errors="ignore")

# Remove index entirely to avoid conflicts
edges = edges.reset_index(drop=True)

# Save CSV
edges.to_csv("data/processed/edges_ndvi_aqi.csv", index=False)

# Save GeoJSON
edges.to_file("data/processed/edges_ndvi_aqi.geojson", driver="GeoJSON")

print("Saved CSV and GeoJSON successfully.")


Preparing for GeoJSON export...
Saved CSV and GeoJSON successfully.


In [20]:
for idx, row in edges.iterrows():
    u, v, k = row["u"], row["v"], row["key"]
    G[u][v][k]["ndvi"] = float(row["ndvi"])
    G[u][v][k]["aqi"] = float(row["aqi"])

ox.save_graphml(G, "data/processed/roads_with_env.graphml")

print("Saved graph → roads_with_env.graphml")


Saved graph → roads_with_env.graphml
